In [ ]:
using Compat
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using BenchmarkTools
using Compat.Test
using RigidBodySim

In [ ]:
# load URDF
mechanism = AtlasRobot.mechanism()
remove_fixed_tree_joints!(mechanism);

In [ ]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [ ]:
# create optimizer
using MathOptInterface
using OSQP.MathOptInterfaceOSQP
const MOI = MathOptInterface
optimizer = OSQPOptimizer()
MOI.set!(optimizer, OSQPSettings.Verbose(), false)
MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [ ]:
# create low level controller
num_basis_vectors = 4
lowlevel = MomentumBasedController{num_basis_vectors}(mechanism, optimizer);
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6 # TODO
        contact.weight[] = 1e-3
    end
end

In [ ]:
# state initialization
function initialize!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    state
end

In [ ]:
# create standing controller
feet = findbody.(mechanism, ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
initialize!(nominalstate)
controller = StandingController(lowlevel, feet, pelvis, nominalstate);

In [ ]:
state = MechanismState(mechanism)
initialize!(state)
τ = similar(velocity(state));
benchresult = @benchmark $controller($τ, 0.0, $state)
@test benchresult.allocs == 0
benchresult

In [ ]:
# function test(τ, state, n)
#     for i = 1 : n
#         controller(τ, 0.0, state)
#     end
# end
# test(τ, state, 1)
# Profile.clear()
# @profile test(τ, state, 10000)
# using ProfileView
# ProfileView.view()

In [ ]:
#NBSKIP
# set up visualizer
using MeshCat
using MeshCatMechanisms
if !isdefined(:vis)
    vis = Visualizer()[:atlas]
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    mvis = MechanismVisualizer(mechanism, visuals, vis)
    set_configuration!(mvis, configuration(nominalstate))
    open(mvis)
    wait(mvis)
end

In [ ]:
# simulate
state = MechanismState(mechanism)
initialize!(state)
Δt = 1 / 500
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, pcontroller)
problem = ODEProblem(dynamics, state, (0., 10.))

In [ ]:
sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
@test sol.retcode == :Success
copyto!(state, last(sol.u))
@test norm(velocity(state)) ≈ 0 atol=1e-8
@test center_of_mass(state).v[3] > 1

In [ ]:
#NBSKIP
setanimation!(mvis, sol)